In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns



from numpy.random import uniform
from numpy.random import normal
from numpy.random import exponential
from numpy.random import lognormal

from numpy.random import choice
from numpy.random import permutation


from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import shapiro

from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

%matplotlib inline

In [2]:
#path='https://raw.githubusercontent.com/Annatkachev/StatLAB/refs/heads/main/DataTrans_1/'
path='/home/anna/PROJECTS2025/StatDatAnLAB/DataTrans/'

## PRED MODEL 1

    1. 
    Consider the dataset'blooddatasubset.csv','bloodsamplesubset.csv', which contain a subset of the blood plasma metabolite dataset (metbolite levels were log2 transformed). Use Lasso regression with indicated parameter to predict the BMI of the inviduals. Don't forget to standardize your features. 
   
    A. Perform a 5-fold cross validation procedure to estimate the mean coefficient of determination R2 accross these folds. Repeat the analysis 5 times, what values do you get?
   
    B. Perform 100 random train-test split (80-20 samples) to estimate the mean coefficient of determination R2. Repeat the analysis 5 times, what values do you get? What is the difference with B? If you want to reduce the variability of this estimate, what parameter could you change?
    
    C. Let's say you had a dataset with 400 features and 80 samples, and you want to compare different classification different models between each other, to chose the one with highest performance (ROC AUC). You perform 5-fold cross-validation and use the mean accross folds to estimate the performance. You obtain ROC AUC estimates 0.92, 0.88, 0.9 and 0.85 for the different models. Explain, why this information is insuficcient to discriminate between the models. 
    
    2. 
    Now consider the same target varialbe, but you will be using a random generated dataset to predict it. What R2 estimate do you obtain for this data? Now perform correlation analysis, to find 25 features with smallest correlation coefficient p-values. Using only 25 features, estimate the R2 values again. What do these results tell you? What rule has been violated in this analysis?
    
    3.
    
    A. Consider the same subset of blood metabolite data. Perform PCA analysis and visualize the difference between female and male samples using color (PC1 and PC2). Now perform statistical analysis to find 20 features with strongest statistical differences between men and women. Perform PCA analysis using only these features from the data table, and visualize the differences between mean and women. Alternitavely, use the PLS regression function to find first and second component of variance that separate men and women, and visualize the first and second components. 
    
    B. What conclusions can you make from these types of analysis? Can you make conclusions about differences in metabolite levels between men and women, using secon and third approach (significant features and PLS regression)?
    
    C. Let's say you added noise to your data (ex. see below). How will the results of your analysis change? What does this tell you? How should PLS regression analysis be used?
    
    4. 
    
    A. Consider the whole metabolite blood dataset 'blooddf.csv','bloodsamples.csv'. Use logistic regression with L1 regularization to predict the sex of the individuals (don't forget to standardize your features). Use different possible parameters C of the model, estimate the ROC AUC value for each value, and visualize the obtained results. What posible range of parameters would give best results for this data?
   
     B. Same as B, but consider the PLS regression, with the number of components as parameter.
   
   
    5.
    
    A. Consider the whole metabolite blood dataset 'bloodsamples.csv', with fixed train and test splits 'sometraindatablood.csv', 'sometestdatablood.csv'. Use logistic regression with L1 regularization to separate male and female samples, with a parameter C such that you get exactly 25 non-zero features in the model trained on the training dataset. What ROC AUC value do you get on the test data?
    
    B. Consider the regression coefficients of the trained model. A higher absolute value indicates a more important feature for the model. Take the feature with highest positive and negative coeficient, and remove them from the data. Retrain your model on the training dataset (you can use the same parameter C). What ROC value do you get for the test data? Next, remove all the 25 non-zero features from the dataset, and repeat this analysis. 
    
    C. From the results of B, what can you say about feature importance for predicting the sex of the samples? Can these 25 features be considered important for predicting sex? Should these 25 features be considered more important than the remaining ones?
    



1

In [3]:
#path='https://raw.githubusercontent.com/Annatkachev/StatLAB/refs/heads/main/PredModel_1/'
path='/home/anna/PROJECTS2025/StatDatAnLAB/PredModel/'

In [4]:
data=pd.read_csv(path+'blooddatasubset.csv',index_col=0)
samples=pd.read_csv(path+'bloodsamplesubset.csv',index_col=0)
lr=Lasso(alpha=0.1)

2

In [5]:
#path='https://raw.githubusercontent.com/Annatkachev/StatLAB/refs/heads/main/PredModel_1/'
path='/home/anna/PROJECTS2025/StatDatAnLAB/PredModel/'

In [6]:
data=pd.read_csv(path+'blooddatasubset.csv',index_col=0)
samples=pd.read_csv(path+'bloodsamplesubset.csv',index_col=0)

datanoise=pd.DataFrame(normal(10,1,(len(data.index),5000)),index=data.index)
Xnorm=(datanoise-datanoise.mean())/datanoise.std()
y=samples.loc[data.index,'bmi']

lr=Ridge(alpha=200,max_iter=1000)

3

In [7]:
#path='https://raw.githubusercontent.com/Annatkachev/StatLAB/refs/heads/main/PredModel_1/'
path='/home/anna/PROJECTS2025/StatDatAnLAB/PredModel/'

In [8]:
data=pd.read_csv(path+'blooddatasubset.csv',index_col=0)
samples=pd.read_csv(path+'bloodsamplesubset.csv',index_col=0)

In [9]:
X=data
samples.loc[samples['sex']=='f','sex binary']=1
samples.loc[samples['sex']=='m','sex binary']=0
y=samples['sex binary']
    
pls = PLSRegression(n_components=2)
pls.fit(X, y)
dfPLS = pd.DataFrame(pls.transform(X),columns=[0,1],index=X.index)

In [10]:
#noise data X:
X=data
for i in X.columns:
    m=X[i].mean()
    s=X[i].std()
    n=len(X.index)

    l=[i+'_x1',i+'_x2',i+'_x3',i+'_x4',i+'_x5',i+'_x6',i+'_x7',i+'_x8']
    X[l]=normal(m,s,(n,8))

/home/anna/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


4

In [11]:
#path='https://raw.githubusercontent.com/Annatkachev/StatLAB/refs/heads/main/DataTrans_1/'
path='/home/anna/PROJECTS2025/StatDatAnLAB/DataTrans/'

data=pd.read_csv(path+'blooddf.csv',index_col=0)
samples=pd.read_csv(path+'bloodsamples.csv',index_col=0)

X=data
samples.loc[samples['sex']=='f','sex binary']=1
samples.loc[samples['sex']=='m','sex binary']=0
y=samples['sex binary']

In [12]:
#param=[...]
#lr=LogisticRegression(penalty='l1', C=param,max_iter=1000,solver='liblinear')

#param=[...]
#pls = PLSRegression(n_components=param)

5

In [17]:
#path='https://raw.githubusercontent.com/Annatkachev/StatLAB/refs/heads/main/PredModel_1/'
path='/home/anna/PROJECTS2025/StatDatAnLAB/PredModel/'
Xtrain=pd.read_csv(path+'sometraindatablood.csv',index_col=0)
Xtest=pd.read_csv(path+'sometestdatablood.csv',index_col=0)

#path='https://raw.githubusercontent.com/Annatkachev/StatLAB/refs/heads/main/DataTrans_1/'
path='/home/anna/PROJECTS2025/StatDatAnLAB/DataTrans/'
samples=pd.read_csv(path+'bloodsamples.csv',index_col=0)
samples.loc[samples['sex']=='f','sex binary']=1
samples.loc[samples['sex']=='m','sex binary']=0

In [18]:
Xtrain_norm=(Xtrain-Xtrain.mean())/Xtrain.std()
Xtest_norm=(Xtest-Xtest.mean())/Xtest.std()
y=samples['sex binary']